# Discrete event simulation
https://docs.netsquid.org/latest-release/tutorial.pydynaa.html

## A quantum ping pong example

In [1]:
import netsquid as ns
import pydynaa
ns.set_random_state(seed=42)

In [2]:
class PingEntity(pydynaa.Entity):
    ping_evtype = pydynaa.EventType('PING_EVENT', 'A ping event')
    delay = 10.

    def start(self, qubit):
        # start the game by scheduling the first ping event after delay
        self.qubit = qubit
        self._schedule_now(PingEntity.ping_evtype)

    def wait_for_pong(self, pong_entity):
        # setup this entity to listen for pong events from a PongEntity
        pong_handler = pydynaa.EventHandler(self._handle_pong_event)
        self._wait(pong_handler, entity=pong_entity, event_type=PongEntity.pong_evtype)

    def _handle_pong_event(self, event):
        # callback function called by the pong handler when pong event is triggered
        m, prob = ns.qubits.measure(self.qubit, observable=ns.Z)
        labels_z = ('|0>', '|1>')
        print(f'{ns.sim_time():.1f}: Pong event! PingEntity measured {labels_z[m]} with probability {prob:.2f}')
        self._schedule_after(PingEntity.delay, PingEntity.ping_evtype)


class PongEntity(pydynaa.Entity):
    pong_evtype = pydynaa.EventType('PONG_EVENT', 'A pong event')
    delay = 10.

    def wait_for_ping(self, ping_entity):
        # setup this entity to listen for ping events from a PingEntity
        ping_handler = pydynaa.EventHandler(self._handle_ping_event)
        self._wait(ping_handler, entity=ping_entity, event_type=PingEntity.ping_evtype)

    def _handle_ping_event(self, event):
        # callback function called by the ping handler when ping event is triggered
        m, prob = ns.qubits.measure(event.source.qubit, observable=ns.X)
        labels_x = ('|+>', '|->')
        print(f"{ns.sim_time():.1f}: Ping event! PongEntity measured {labels_x[m]} with probability {prob:.2f}")
        self._schedule_after(PongEntity.delay, PongEntity.pong_evtype)


In [3]:
# create entities and register them to each other
ping = PingEntity()
pong = PongEntity()
ping.wait_for_pong(pong)
pong.wait_for_ping(ping)

qubit, = ns.qubits.create_qubits(1)
ping.start(qubit)

In [4]:
stats = ns.sim_run(end_time=200)

0.0: Ping event! PongEntity measured |+> with probability 0.50
10.0: Pong event! PingEntity measured |1> with probability 0.50
20.0: Ping event! PongEntity measured |-> with probability 0.50
30.0: Pong event! PingEntity measured |1> with probability 0.50
40.0: Ping event! PongEntity measured |+> with probability 0.50
50.0: Pong event! PingEntity measured |0> with probability 0.50
60.0: Ping event! PongEntity measured |+> with probability 0.50
70.0: Pong event! PingEntity measured |1> with probability 0.50
80.0: Ping event! PongEntity measured |-> with probability 0.50
90.0: Pong event! PingEntity measured |1> with probability 0.50
100.0: Ping event! PongEntity measured |+> with probability 0.50
110.0: Pong event! PingEntity measured |1> with probability 0.50
120.0: Ping event! PongEntity measured |-> with probability 0.50
130.0: Pong event! PingEntity measured |0> with probability 0.50
140.0: Ping event! PongEntity measured |+> with probability 0.50
150.0: Pong event! PingEntity measur

In [6]:
print(stats)


Simulation summary

Elapsed wallclock time: 0:00:00.005043
Elapsed simulation time: 2.00e+02 [ns]
Triggered events: 20
Handled callbacks: 20
Total quantum operations: 20
Frequent quantum operations: MEASURE = 20
Max qstate size: 1 qubits
Mean qstate size: 1.00 qubits



## Event expressions by example: quantum teleportation

In [49]:
ns.sim_reset()

In [50]:
class Charlie(pydynaa.Entity):
    ready_evtype = pydynaa.EventType('QUBITS_READY', 'Entangled qubits are ready')
    _generate_evtype = pydynaa.EventType('GENERATE', 'Generate entangled qubits')
    period = 60.  # the cycle, i.e., every 60 ns
    delay = 10.
    
    def __init__(self):
        # initialise Charlie by entangling qubits after every generation event
        self.entangled_qubits = None
        self._generate_handler = pydynaa.EventHandler(self._entangle_qubits)
        self._wait(self._generate_handler, entity=self, event_type=Charlie._generate_evtype)
        
    def _entangle_qubits(self, event):
        # callback function that entangles qubits and schedules an entanglement ready event
        q1, q2 = ns.qubits.create_qubits(2)
        ns.qubits.operate(q1, ns.H)
        ns.qubits.operate([q1, q2], ns.CNOT)
        self.entangled_qubits = [q1, q2]
        self._schedule_after(Charlie.delay, Charlie.ready_evtype)
        print(f'{ns.sim_time():.1f}: Charlie finished generating entanglement')
        self._schedule_after(Charlie.period, Charlie._generate_evtype)
    
    def start(self):
        # begin generating entanglement
        print(f'{ns.sim_time():.1f}: Charlie start generating entanglement')
        self._schedule_now(Charlie._generate_evtype)


In [51]:
class Alice(pydynaa.Entity):
    ready_evtype = pydynaa.EventType('CORRECTION_READY', 'Corrections are ready')
    _teleport_evtype = pydynaa.EventType('TELEPORT', 'Teleport the qubit')
    delay = 20.
    
    def __init__(self, teleport_state):
        # initialise alice by setting the teleport state and waiting to teleport
        self.teleport_state = teleport_state
        self.q0 = None
        self.q1 = None
        self.corrections = None
        self._teleport_handler = pydynaa.EventHandler(self._handle_teleport)
        self._wait(self._teleport_handler, entity=self, event_type=Alice._teleport_evtype)
    
    def wait_for_charlie(self, charlie):
        # setup alice to wait for an entanglement qubit from charlie
        self._qubit_handler = pydynaa.EventHandler(self._handle_qubit)
        self._wait(self._qubit_handler, entity=charlie, event_type=Charlie.ready_evtype)
    
    def _handle_qubit(self, event):
        # callback function that handles arrival of entangled qubit and schedules teleportation
        self.q0, = ns.qubits.create_qubits(1, no_state=True)
        self.q1 = event.source.entangled_qubits[0]
        ns.qubits.assign_qstate([self.q0], self.teleport_state)
        self._schedule_after(Alice.delay, Alice._teleport_evtype)
        print(f'{ns.sim_time():.1f}: alice received entangled qubit')
    
    def _handle_teleport(self, event):
        # callback function that does teleportion and schedules a correction ready event
        ns.qubits.operate([self.q0, self.q1], ns.CNOT)
        ns.qubits.operate(self.q0, ns.H)
        m0, _ = ns.qubits.measure(self.q0)
        m1, _ = ns.qubits.measure(self.q1)
        self.corrections = [m0, m1]
        self._schedule_now(Alice.ready_evtype)
        print(f'{ns.sim_time():.1f}: alice measured qubits & sending  corrections')


In [52]:
class Bob(pydynaa.Entity):
    def wait_for_teleport(self, alice, charlie):
        # setup bob to wait for his entangled qubit and alice's corrections
        charlie_ready_evexpr = pydynaa.EventExpression(source=charlie, event_type=Charlie.ready_evtype)
        alice_ready_evexpr = pydynaa.EventExpression(source=alice, event_type=Alice.ready_evtype)
        both_ready_evexpr = charlie_ready_evexpr & alice_ready_evexpr
        self._teleport_handler = pydynaa.ExpressionHandler(self._handle_teleport)
        self._wait(self._teleport_handler, expression=both_ready_evexpr)
        # you wait for the event. when the event indeed occurs, it is the callback function's (which is a handler) show time

    def _handle_teleport(self, event_expression):
        # callback function that handles message from both alice and charlie
        qubit = event_expression.first_term.atomic_source.entangled_qubits[1]
        alice = event_expression.second_term.atomic_source
        self._apply_corrections(qubit, alice.corrections)
    
    def _apply_corrections(self, qubit, corrections):
        # apply teleportation corrections and check fidelity
        m0, m1 = corrections
        if m1:
            ns.qubits.operate(qubit, ns.X)
        if m0:
            ns.qubits.operate(qubit, ns.Z)
        fidelity = ns.qubits.fidelity(qubit, alice.teleport_state)
        print(f"{ns.sim_time():.1f}: Bob received entangled qubit and corrections! Fidelity = {fidelity:.3f}")

In [53]:
class NoisyBob(Bob):
    depolar_rate = 1e7

    def _handle_teleport(self, event_expression):
        # callback function that first applies noise to qubit before corrections
        alice_expr = event_expression.second_term
        charlie_expr = event_expression.first_term
        # compute time that qubit from charlie has been waiting
        delay = ns.sim_time() - charlie_expr.triggered_time
        # apply time-dependent quantum noise to bob's qubit
        qubit = charlie_expr.atomic_source.entangled_qubits[1]
        ns.qubits.delay_depolarize(qubit, NoisyBob.depolar_rate, delay)
        # apply classical corrections (as before)
        self._apply_corrections(qubit, alice_expr.atomic_source.corrections)


In [54]:
def setup_network(alice, bob, charlie):
    alice.wait_for_charlie(charlie)
    bob.wait_for_teleport(alice, charlie)
    charlie.start()

alice = Alice(teleport_state=ns.h1)
bob = Bob()
charlie = Charlie()

In [55]:
setup_network(alice, bob, charlie)

0.0: Charlie start generating entanglement


In [56]:
stats = ns.sim_run(end_time=200)

0.0: Charlie finished generating entanglement
10.0: alice received entangled qubit
30.0: alice measured qubits & sending  corrections
30.0: Bob received entangled qubit and corrections! Fidelity = 1.000
60.0: Charlie finished generating entanglement
70.0: alice received entangled qubit
90.0: alice measured qubits & sending  corrections
90.0: Bob received entangled qubit and corrections! Fidelity = 1.000
120.0: Charlie finished generating entanglement
130.0: alice received entangled qubit
150.0: alice measured qubits & sending  corrections
150.0: Bob received entangled qubit and corrections! Fidelity = 1.000
180.0: Charlie finished generating entanglement
190.0: alice received entangled qubit


In [27]:
print(stats)


Simulation summary

Elapsed wallclock time: 0:00:00.001874
Elapsed simulation time: 1.00e+02 [ns]
Triggered events: 8
Handled callbacks: 10
Total quantum operations: 13
Frequent quantum operations: MEASURE = 4; H = 4; CX = 4; Z = 1
Max qstate size: 3 qubits
Mean qstate size: 2.23 qubits



In [39]:
ns.sim_reset()
ns.set_qstate_formalism(ns.QFormalism.DM)

In [40]:
alice = Alice(teleport_state=ns.h1)
bob = NoisyBob()
charlie = Charlie()
setup_network(alice, bob, charlie)

0.0: Charlie start generating entanglement


In [41]:
stats = ns.sim_run(end_time=50)

0.0: Charlie finished generating entanglement
10.0: alice received entangled qubit
30.0: alice measured qubits & sending  corrections
30.0: Bob received entangled qubit and corrections! Fidelity = 0.954
